In [184]:
from transformers import AutoTokenizer, RobertaForQuestionAnswering, TrainingArguments, Trainer
from datasets import load_dataset
import torch
import time
import torch.nn as nn

In [185]:
tokenizer = AutoTokenizer.from_pretrained("deepset/roberta-base-squad2")
model = RobertaForQuestionAnswering.from_pretrained("deepset/roberta-base-squad2")

In [186]:
train_data = load_dataset("csv", data_files="Dados/train_data.csv")
test_data = load_dataset("csv", data_files="Dados/test_data.csv")

print(train_data['train'])

Dataset({
    features: ['context', 'question', 'answer', 'start'],
    num_rows: 41
})


In [187]:
def tokenize_function(example):
    context_prompt = 'context: '
    question_prompt = 'question: '
    answer_human_prompt = ' human_answer: '
    answer_model_prompt = ' model_answer: '
    prompts = [context_prompt + context + question_prompt + question + answer_human_prompt for context, question in zip(example["context"], example["question"])]
    respostas = [answer + answer_model_prompt for answer in example["answer"]]
    inputs = tokenizer(prompts, padding="max_length", truncation=True, return_tensors="pt")
    labels = tokenizer(respostas, padding="max_length", truncation=True, return_tensors="pt")
    example['input_ids'] = inputs.input_ids
    example['attention_mask'] = inputs.attention_mask

    return example

tokenized_train_data = train_data.map(tokenize_function, batched=True)
tokenized_test_data = train_data.map(tokenize_function, batched=True)

Map:   0%|          | 0/41 [00:00<?, ? examples/s]

In [188]:
print(tokenized_train_data['train'][5])

{'context': 'O Aprendiz do Futuro é uma iniciativa do Governo do Estado de Goiás, por meio da Secretaria de Estado de Desenvolvimento Social (SEDS), que tem como visão oferecer o maior programa de sócio-aprendizagem do Brasil e do mundo com foco em tecnologia, alta performance e impacto social na vida dos jovens em situação de vulnerabilidade.', 'question': 'Quais a visão do programa Aprendiz do Futuro?', 'answer': 'A visão do Programa Aprendiz do Futuro é criar um ambiente de aprendizado inovador e inclusivo, onde os jovens possam desenvolver suas habilidades e alcançar seu pleno potencial, contribuindo assim para um futuro melhor para si e para suas comunidades.', 'start': 0, 'input_ids': [0, 46796, 35, 384, 83, 5234, 1187, 1210, 109, 13611, 7367, 7935, 1717, 1916, 11, 636, 11284, 7222, 109, 22595, 139, 109, 5441, 2102, 263, 2381, 118, 6417, 6, 2953, 162, 1020, 2955, 8350, 8954, 263, 5441, 2102, 263, 4762, 225, 13728, 705, 8913, 139, 3574, 36, 104, 1691, 104, 238, 1192, 11030, 3137, 

In [189]:
tokenizer.decode(tokenized_test_data['train'][5]['input_ids'], skip_special_tokens=True)

'context: O Aprendiz do Futuro é uma iniciativa do Governo do Estado de Goiás, por meio da Secretaria de Estado de Desenvolvimento Social (SEDS), que tem como visão oferecer o maior programa de sócio-aprendizagem do Brasil e do mundo com foco em tecnologia, alta performance e impacto social na vida dos jovens em situação de vulnerabilidade.question: Quais a visão do programa Aprendiz do Futuro? human_answer: '

In [193]:
training_args = TrainingArguments(
    output_dir=output_dir,
    learning_rate=1e-5,
    num_train_epochs=1,
    weight_decay=0.01,
    logging_steps=1,
    max_steps=1
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_data['train'],
    eval_dataset=tokenized_test_data['train']
)


In [194]:
trainer.train()

ValueError: The model did not return a loss from the inputs, only the following keys: start_logits,end_logits. For reference, the inputs it received are input_ids,attention_mask.